In [1]:
# Generate data to infer the rules for integral

import json
import numpy as np
import sympy as sp
from sympy import sympify, lambdify, symbols, integrate, Interval, Symbol, I, S, oo, plot, evalf, N
from IPython.display import display
from utils.utils import *


# def remove_constants(f):
#     t = Symbol('t')
#     return f.as_independent(t)[1]

# fin = open("datasets/parametric_equations_polynomial_integral_results.json", "r")
# lines = fin.readlines()
# fin.close()
# fin = open("datasets/parametric_equations_randomized_polynomial_integral_results.json", "r")
# lines.extend(fin.readlines())
# fin.close()

# MAX_POWER = 6
# MAX_AVG_DIFF = 0.01

# originals = []
# integrals = []
# t = Symbol('t')

# for line in lines:
#     result = json.loads(line)
#     if "rounded_regressed" not in result:
#         continue
#     original = round_all_floats(N(sympify(result["original"])))
#     integral = remove_constants(round_all_floats(N(sympify(result["rounded_regressed"]))))
#     try:
#         original = filter_non_polynomial(original)
#         integral = filter_non_polynomial(integral)
#         original_integral = integrate(original, t)
#         avg_diff = get_avg_diff(original_integral, integral, t)
#         if avg_diff > MAX_AVG_DIFF or len(original_integral.args) != len(integral.args):
#             print("Skipping. Diff=", avg_diff)
#             display(original_integral)
#             display(integral)
#             continue
#     except:
#         print("Cannot filter non-polynomials on", str(integral))
#         continue
#     originals.append(str(original))
#     integrals.append(str(integral))
#     if len(originals) % 100 == 0:
#         print(len(originals), "cases loaded")
    
# fin.close()

# sentences = []
# for i in range(len(originals)):
#     sentences.append(originals[i] + ' entail ' + integrals[i] + ' end')
# fin.close()



In [2]:
import json
import torch
import random
from datasets import load_dataset, Dataset, load_from_disk

# random.shuffle(sentences)

# ds = Dataset.from_dict({'eq_pair': sentences})
# train_ds = ds.train_test_split(test_size=0.025)

# train_ds['train'][1]

# train_ds.save_to_disk("datasets/integrate_gptneo_dataset_50k")

train_ds = load_from_disk("datasets/integrate_gptneo_dataset_50k")

/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
train_ds

DatasetDict({
    train: Dataset({
        features: ['eq_pair'],
        num_rows: 47881
    })
    test: Dataset({
        features: ['eq_pair'],
        num_rows: 1228
    })
})

In [4]:
from transformers import AutoTokenizer

CONTEXT_LENGTH = 256

tokenizer = AutoTokenizer.from_pretrained("xhyi/PT_GPTNEO350_ATG") #"EleutherAI/gpt-neo-125m") 
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def preprocess_function(examples):
    return tokenizer(examples["eq_pair"], padding='max_length', truncation=True, max_length=CONTEXT_LENGTH, return_tensors="pt")

tokenized_ds = train_ds.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=train_ds["train"].column_names,
)

def preprocess_function(examples):
    examples["labels"] = examples["input_ids"].copy()
    return examples

lm_dataset = tokenized_ds.map(preprocess_function, batched=True, num_proc=1)

from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [5]:
# for i in range(len(lm_dataset['train'])):
#     if len(lm_dataset['train'][i]['input_ids']) != 256:
#         print(i, len(lm_dataset['train'][i]['input_ids']))
#     if len(lm_dataset['train'][i]['labels']) != 256:
#         print(i, len(lm_dataset['train'][i]['labels']))

In [6]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("xhyi/PT_GPTNEO350_ATG")  #"EleutherAI/gpt-neo-125m")
# config = model0.config

# model = AutoModelForCausalLM.from_config(config)
# model0 = None

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="datasets/integrate_gptneo_350m_50k_202401",
    evaluation_strategy="steps",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    fp16=True,
    save_steps=500,
    eval_steps=500,
    logging_steps=500,
    save_total_limit=4,
    push_to_hub=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

trainer.train()

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


In [ ]:
#model.save_pretrained("datasets/integrate_gptneo_202401/gptneo-350m-14000.model")

In [ ]:
# device = "cuda:0" if torch.cuda.is_available() else "cpu"

# model = AutoModelForCausalLM.from_pretrained("datasets/integrate_gptneo_202401/gptneo-350m-5500-loss0.350.model").to(device)

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

def generate_integral(input):
    # Encode some input text
    prompt = input + " entail"
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
    # Generate text
    output = model.generate(input_ids, max_length=100, num_return_sequences=1, temperature=0.7)
    # Decode and print the output
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    #print(generated_text)
    if 'entail' in generated_text:
        generated_text = generated_text[generated_text.find('entail') + 6:].strip()
        #print(generated_text)
    if 'end' in generated_text:
        generated_text = generated_text[0:generated_text.find('end')].strip()
    return generated_text
    
print(generate_integral("2.5*t**3 + 0.51*t**2 + 3.2*t + 1.2"))

In [ ]:
train_ds['test'][0]

In [ ]:
import sympy as sp
from utils.utils import *

MIN_ALLOWED_DIFF = 0.011

test_ds = train_ds['test']

num_processed = 0
num_equal = 0
num_zero_diff = 0
num_within_allowed = 0
t = sp.Symbol('t')

verbose = False

for i in range(min(len(test_ds), 1000)):
    if verbose:
        print("Case", i, test_ds[i]['eq_pair'])
    eq_pair = test_ds[i]['eq_pair']
    question = eq_pair[0:eq_pair.find('entail')].strip()
    answer = eq_pair[eq_pair.find('entail') + 6:].replace('end', '').strip()
    if 'repeat' in question:
        question = question[0:question.find('repeat')]
    original = sp.sympify(question)
    integral = round_all_floats(sp.integrate(original), 2)
    if verbose: display(integral)
    try:
        pred = generate_integral(question)
        generated = round_all_floats(sp.sympify(pred), 2)
        if verbose: display(generated)
    except:
        print("Cannot sympify", pred)
        continue
    avg_diff = get_avg_diff(integral, generated, t)
    if verbose: print("avg_diff", avg_diff)
    num_processed += 1
    if avg_diff <= MIN_ALLOWED_DIFF:
        num_within_allowed += 1
    else:
        display(integral)
        display(generated)
        print(avg_diff)
    if avg_diff <= 0.0000001:
        num_zero_diff += 1
    diff_expr = sp.simplify(integral-generated)
    if verbose: display(diff_expr)
    if diff_expr == 0:
        if verbose: print("Equal")
        num_equal += 1
    if i%100 == 0:
        print(i+1, "processed")
        
print("num_processed", num_processed)
print("num_equal", num_equal)
print("num_zero_diff", num_zero_diff)
print("num_within_allowed", num_within_allowed)